---
layout: post
title: Speeding up CRC-32 calculations in Mojo 
categories: [mojo]
date: "2024-05-99"
author: "Ferdinand Schenck"
draft: true
description: Or how I made CRC-32 calculations AA times faster than Python, and BB times slower than Python. 
---

In my [last post](https://fnands.com/blog/2024/mojo-png-parsing/) on parsing PNG images in I very briefly mentioned cyclic redundancy checks, and posted a rather cryptic looking function which I claimed was a bit inefficient. 

In this post I want to follow up on that a bit and see how we can speed up these calculations, including a little bit of a look at the compile-time metaprogramming side of Mojo.    

For reference, this post was done with Mojo 24.4.0, so a few language details have changed since my last post (e.g. `math.bit` got moved to the top-level `bit` and a few of it's functions have been renamed). 


## A bit of context
But first, let's go through a bit of background so we know what we're dealing with. 

### Cyclic redundancy checks

CRCs are error detecting codes that are often used to detect corruption of data in digital files, an example of which is PNG files. In the case of PNGs for example the CRC-32 is calculated for the data of each chunk and appended to the end of the chunk, so that the person reading the file can verify whether the data they read was the same as the data that was written.  

A CRC check technically does "long division in the ring of polynomials of binary coefficients ($\Bbb{F}_2[x]$)" 😳.   

It's not as complicated as it sounds. I found the [Wikipedia article on Polynomial long division](https://en.wikipedia.org/wiki/Polynomial_long_division) to be helpful, and if you want an in depth explanation then
[this post](https://github.com/komrad36/CRC) by [Kareem Omar](https://github.com/komrad36) does an excellent job of explaining both the concept and implementation considerations. I won't go deep into the explanations, so I recommend you read at least the first part of Kareem's post for more background. I pretty much use his post as a guide. 

Did you read that post? Then welcome back, and well continue from there. 

But tl;dr: 
XOR is equivalent to polynomial long division (over a finite field) for binary numbers, and XOR is a very efficient operation to calculate in hardware. 
Essentially what a CRC check does in practice is to run through a sequence of bytes, and iteratively perform a lot of XORs and bit-shifts. By iteratively going through each bit, one can come up with a value that will (nearly certainly) change if data is corrupted somehow. 

## The original 

The CRC-32 check from my previous post looked something like this: 

In [2]:
from bit import bit_reverse

fn CRC32(data: List[UInt8], dummy_table: List[UInt32]) -> UInt32:
    var crc32: UInt32 = 0xffffffff
    for byte in data:
        crc32 = (bit_reverse(byte[]).cast[DType.uint32]() << 24) ^ crc32
        for _ in range(8):
            
            if crc32 & 0x80000000 != 0:
                crc32 = (crc32 << 1) ^ 0x04c11db7
            else:
                crc32 = crc32 << 1
                
    return bit_reverse(crc32^0xffffffff)

I'll step through this in a moment, but the first thing you might notice here is that I am reversing a lot of bits here. The table argument is a bit of future proofing, which we won't need for now, but will become apparent soon. Just ignore it for now. 


This is because when I was implementing this function (based off a C example), I implemented a little-endian version of the algorithm, while PNGs are encoded as big-endian. It's not a huge deal, but it does mean that I am constantly reversing bytes, and then reversing the output again.

## The correct bit order


We can make this better by implementing the big-endian version:

In [3]:
fn CRC32_inv(data: List[UInt8], dummy_table: List[UInt32]) -> UInt32:
    """Big endian CRC-32 check using 0xedb88320 as polynomial."""

    # Initialize crc32 as all 1s
    var crc32: UInt32 = 0xffffffff

    # Step though all bytes in bytestream
    for byte in data:

        # XOR new byte with crc32 
        # We need to cast it to a UInt32 first as Mojo doesn't do implicit casts
        crc32 = (byte[].cast[DType.uint32]() ) ^ crc32

        # Step though crc32 8 times
        for _ in range(8):
            
            # If leading bit is 1, bitshift by 1 and XOR with the generator polynomial
            # otherwise just shift by one 
            if crc32 & 1 != 0:
                crc32 = (crc32 >> 1) ^ 0xedb88320
            else:
                crc32 = crc32 >> 1

    # Invert upon return
    return crc32^0xffffffff

 This is very similar, and just entails that we use a the reverse of the polygon we did before (if you bit reverse `0x04c11db7` you get `0xedb88320`). This also saves us one 24-bit bit-shift, as we are now working on the bottom 8 bits of the `UInt32` instead of the top 8. 

 Just to verify that these implementations are equivalent, let's do a quick test: 

In [4]:
var example_list = List[UInt8](5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201,
                                           5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201,
                                           5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201,
                                           5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201, 
                                           5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201,
                                           5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201,
                                           5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201,
                                           5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201, 42)

print(hex(CRC32(example_list, List[UInt32](1))))
print(hex(CRC32_inv(example_list, List[UInt32](1))))                                           

0x382aa34e
0x382aa34e


And there we go, a more elegant version of the CRC-32 check I implemented last time.  

As the theme of today's post is trying to speed things up, let's do a little bit of benchmarking to see if this change has saved us any time. As we are doing one fewer bit reverse and bit shift per byte, as well as a final reverse, we should see a bit of a performance uplift. 


## A bit of benchmarking:


So let's define a benchmarking function. This function will take two version of the CRC32 function and benchmark their runtimes. 

We have to a little bit of work first. The functions we test need to return `None`, so we need to wrap our functions in functions with no return value. 
Note the `benchmark.keep(a)` calls: the compiler will realize that `a` is never used and will compile this away unless you instruct it to keep them: 

In [5]:
import benchmark
from testing import assert_equal

alias data_table_func = fn(data: List[UInt8], table: List[UInt32]) -> UInt32


fn wrap_func[testing_function: data_table_func, data: List[UInt8], table: List[UInt32] ]():
    var a =  testing_function(data = data, table = table)
    benchmark.keep(a)


fn bench[function_1: data_table_func, function_2: data_table_func,
         test_list: List[UInt8], test_table: List[UInt32]]() raises -> Tuple[Float64, Float64]:


    var CRC32_1 = function_1(test_list, test_table) 

    alias function_1_wrapped = wrap_func[function_1]
    var report_1 = benchmark.run[function_1_wrapped[test_list, test_table]](max_runtime_secs=1.0
    ).mean(benchmark.Unit.ms)
    print("Function 1 runtime (ms): \t", report_1)


    var CRC32_2 = function_2(test_list, test_table) 
    
    #assert_equal(CRC32_1, CRC32_2)

    alias function_2_wrapped = wrap_func[function_2]
    var report_2 = benchmark.run[function_2_wrapped[test_list, test_table]](max_runtime_secs=1.0
    ).mean(benchmark.Unit.ms)
    print("Function 2 runtime (ms): \t", report_2)

    print("Speedup factor: \t\t", (report_1/report_2))

    return report_1, report_2

Next we need a test case. I'm not sure if there is a nicer way to fill a List with random values yet, but for now we can list create an `UnsafePointer`, alloc some space, and fill it with random numbers using `rand`. 
Then we can init a `List` using that data: 

In [6]:
from random import rand

alias fill_size = 2**20
alias g = UnsafePointer[UInt8].alloc(fill_size)
rand[DType.uint8](ptr =  g, size = fill_size)
alias rand_list = List[UInt8](unsafe_pointer = g, size = fill_size, capacity = fill_size)

alias dummy_table = List[UInt32](1)

And finally we are ready to benchmark: 

In [12]:
original_time, improved_time = bench[CRC32, CRC32_inv, rand_list, dummy_table]()

Function 1 runtime (ms): 	 10.702447675
Function 2 runtime (ms): 	 7.478882585
Speedup factor: 		 1.4310222888731179


Nice! So just by avoiding a few bit reversals and bit shifts we get about a 40% uplift in performance depending on the run. 

**Note:** I am doing this in a Jupyter notebook, so there is a bit of variance from run to run. 


## A quick Python speed-check

While we're checking performance, let's see how this implementation would perform in Python: 

In [14]:
%%python

def py_crc32(data: bytearray) -> int: 
    crc32 = 0xffffffff
    
    for byte in data:
        crc32 = byte ^ crc32

        for i in range(8):
            if crc32 & 1 != 0:
                crc32 = (crc32 >> 1) ^ 0xedb88320
            else:
                crc32 = crc32 >> 1

    return crc32^0xffffffff

And let's just do a sanity check to assure ourselves that we produce the same CRC-32 value given the same bytestream: 

In [15]:
%%python

py_example_list = [5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201,
    5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201,
    5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201,
    5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201, 
    5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201,
    5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201,
    5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201,
    5, 78, 138, 1, 54, 172, 104, 99, 54, 167, 94, 56, 22, 184, 204, 90, 201, 42]
    
py_example_bytearray = bytearray(py_example_list)

print(hex(py_crc32(py_example_bytearray)))

0x382aa34e


In [16]:
%%python
from timeit import timeit
import numpy as np


py_rand_list = np.random.randint(0, 255, size=2**20, dtype=np.uint8)

naive_python_seconds = timeit(lambda: py_crc32(py_rand_list), number=10)/10

print(f"Naive Python runtime (ms): {naive_python_seconds*100}")

Naive Python runtime (ms): 137.24778024014086


That's pretty slow in fact this means that: 

In [17]:
print("Speedup over Python:", (naive_python_seconds*100/improved_time), "slower")

Speedup over Python: 18.351375179416706 slower


So that's a nice little speedup we can get by writing essentially the same logic in Mojo. 

Now this is a bit unfair. If you actually wrote the this function in Python for anything other than educational purposes, then you are using Python wrong, but I'll get back to how you would actually do it later. 



## Adding a table

Now, the majority of CRC-32 implementations you see will usually use a table. 

This is called the [Sarwate algorithm](https://en.wikipedia.org/wiki/Computation_of_cyclic_redundancy_checks#Sarwate_algorithm_(single_lookup_table)) and exploits the fact that as we are operating on data one byte at a time, there are only 256 unique values that the calculation in the inner loop of the algorithm can take. This is a small enough table (1024 bytes for 256 unsigned 32-bit integers) that it can easily be kept in the L1 cache during computation. 

Let's try it out to see how much faster this get's us.

First, we need to pre-compute the table: 

In [19]:

fn fill_table() -> List[UInt32]:
    var table = List[UInt32](capacity=256)
    for i in range(256):

        var key = UInt8(i)
        var crc32 = key.cast[DType.uint32]()
        for i in range(8):
            if crc32 & 1 != 0:
                crc32 = (crc32 >> 1) ^ 0xedb88320
            else:
                crc32 = crc32 >> 1

        table[i] = crc32
    return table

alias little_endian_table = fill_table()


We've now effectively amortized the cost of the inner loop and replaced it with a table lookup which should in principle be faster. 

Our CRC-32 algorithm now take this form: 

In [20]:
fn CRC32_table(data: List[UInt8], table: List[UInt32]) -> UInt32:
    var crc32: UInt32 = 0xffffffff
    
    for byte in data:
        var index = (crc32 ^ byte[].cast[DType.uint32]()) & 0xff
        crc32 = table[int(index)] ^ (crc32 >> 8)

    return crc32^0xffffffff

# Sanity check
print(hex(CRC32(example_list, List[UInt32](1))))
print(hex(CRC32_table(example_list, little_endian_table)))                                           

0x382aa34e
0x382aa34e


Now we can test it out! 

In [21]:
_, _ = bench[CRC32, CRC32_table, rand_list, little_endian_table]()

Function 1 runtime (ms): 	 10.878752649999999
Function 2 runtime (ms): 	 2.117795905511811
Speedup factor: 		 5.1368276903769505


So a speedup of around 5 times as fast as when we started. This is already a pretty good result, but can we do better? 

## Two byte table

In principle, we could load the data as a 16-bit integer at a time and use the same trick above to build a table. However, such a table would have 65536 entries, resulting in a 256 KB table, which is a lot bigger than the 48 KB of L1 cache on my machine. 


It turns out, we can still [do two bytes at a time](https://en.wikipedia.org/wiki/Computation_of_cyclic_redundancy_checks#Byte-Slicing_using_multiple_tables).  

Following the [description given by Kareem Omar](https://github.com/komrad36/CRC?tab=readme-ov-file#option-6-1-byte-tabular), we realize that if we have two successive bytes in our message, let's call them `X` and `Y`: 

```
X = xxxxxxxx
Y = yyyyyyyy
```

if we pad these messages with zeroes

```
Xz = xxxxxxxx00000000
zY = 00000000yyyyyyyy
```

then 

```
Xz ^ zY = xxxxxxxxyyyyyyyy = M
```

What this means is that if we have a CRC algorithm that works on a 16-bit message called M then: 

```
CRC(M) = CRC(Xz ^ zY) = CRC(Xz) ^ CRC(zY) 
```

For the operation on `Y`, leading zeros don't affect the calculation, so we can just use the 8-bit CRC algorithm we have developed above: 

```
CRC(zY) = CRC(Y)
```

The same does not hold for trailing zeroes. However, as `Xz` will always be of the form `xxxxxxxx00000000` and there can only be 256 unique values for `Xz`, we can build a new table with 256 entries to look up these values. We can build two separate 256 entry tables, or we can just build one 512 entry table, 


So let's construct this new table: 

In [23]:
fn fill_table_2_byte() -> List[UInt32]:

    var table = List[UInt32](capacity=512)
    table.size = 512
    table.resize(256*2, 0)

    for i in range(256):

        var key = UInt8(i)
        var crc32 = key.cast[DType.uint32]()
        for i in range(8):
            if crc32 & 1 != 0:
                crc32 = (crc32 >> 1) ^ 0xedb88320
            else:
                crc32 = crc32 >> 1

        table[i] = crc32

    for i in range(256, 512):
        var crc32 = table[i-256]
        table[i] = (crc32 >> 8) ^ table[int(crc32.cast[DType.uint8]())]
    return table

var little_endian_table_2_byte = fill_table_2_byte()


We have to now update our algorithm to do two table lookups instead of one. 

Additionally, we have to modify our algorithm to read the data two bytes at a time. 



In [24]:
fn CRC32_table_2_byte(data: List[UInt8], table: List[UInt32]) -> UInt32:
    var crc32: UInt32 = 0xffffffff

    var length = len(data)//2
    var extra = len(data) % 2

    # Go through range in steps of 2
    for i in range(start = 0, end = length * 2 , step = 2):
        
        # Concatenate the two bytes
        var val: UInt32 = ((data[i + 1].cast[DType.uint32]() << 8) | data[i].cast[DType.uint32]())
        # Creat index, and look up values in table
        var index = crc32 ^ val
        crc32 =  table[int((index >> 8).cast[DType.uint8]())] ^ table[256 + int(index.cast[DType.uint8]())] ^ (crc32 >> 16)

    # Tail call to calculate final byte in case message not divisable by 2
    for i in range(2*length, 2*length + extra ):
        var index = (crc32 ^ data[i].cast[DType.uint32]()) & 0xff
        crc32 = table[int(index)] ^ (crc32 >> 8)


    return crc32^0xffffffff

# Sanity check
print(hex(CRC32(example_list, List[UInt32](1))))
print(hex(CRC32_table_2_byte(example_list, little_endian_table_2_byte)))

0x382aa34e
0x382aa34e


So let's see if this speeds things up: 

In [25]:
alias a_little_endian_table_2_byte = fill_table_2_byte()


_, _ = bench[CRC32, CRC32_table_2_byte, rand_list, a_little_endian_table_2_byte]()

Function 1 runtime (ms): 	 11.027001589999999
Function 2 runtime (ms): 	 1.4018122798766384
Speedup factor: 		 7.8662469635166792


Nice, now we are at a nearly 8x speedup to where we began.

But can we go further? There's nothing in the above that forces us to only use two bytes at a time. 



## 4-bytes tabular

Nothing stops us from doing 4 bytes, i.e. 32-bits at a time with similar logic to above. 

I'll quickly create a function that will fill an arbitrarily sized table: 

In [27]:
fn fill_table_n_byte[n: Int]() -> List[UInt32]:
    var table = List[UInt32](capacity=256 * n)
    table.size = 256*n

    for i in range(256 * n):
        if i < 256:
            var key = UInt8(i)
            var crc32 = key.cast[DType.uint32]()
            for _ in range(8):
                if crc32 & 1 != 0:
                    crc32 = (crc32 >> 1) ^ 0xEDB88320
                else:
                    crc32 = crc32 >> 1

            table[i] = crc32
        else:
            var crc32 = table[i - 256]
            var index = int(crc32.cast[DType.uint8]())
            table[i] = (crc32 >> 8) ^ table[index]

    return table

var little_endian_table_4_byte = fill_table_n_byte[4]()

Now to create the 4-byte function: 

In [28]:
fn CRC32_table_4_byte(data: List[UInt8], table: List[UInt32]) -> UInt32:
    var crc32: UInt32 = 0xffffffff

    var size = 4

    var length = len(data)//size
    var extra = len(data) % size

    for i in range(start = 0, end = length*size, step = size):
        
        var val: UInt32 =  (data[i + 3].cast[DType.uint32]() << 24) | (data[i + 2].cast[DType.uint32]() << 16) | (data[i + 1].cast[DType.uint32]() << 8) | data[i].cast[DType.uint32]()
        var index = crc32 ^ val.cast[DType.uint32]()
        crc32 = table[0*256 + int((index >> 24).cast[DType.uint8]())] ^
                table[1*256 + int((index >> 16).cast[DType.uint8]())] ^
                table[2*256 + int((index >> 8).cast[DType.uint8]())] ^
                table[3*256 + int((index >> 0).cast[DType.uint8]())] 
    
    for i in range(size*length, size*length + extra ):
        var index = (crc32 ^ data[i].cast[DType.uint32]()) & 0xff
        crc32 = table[int(index)] ^ (crc32 >> 8)


    return crc32^0xffffffff

# Sanity check
print(hex(CRC32(example_list, List[UInt32](1))))
print(hex(CRC32_table_4_byte(example_list, little_endian_table_4_byte)))

0x382aa34e
0x382aa34e


And presto, it still works!

Let's see how much faster we are now? 



In [29]:
alias a_little_endian_table_4_byte = fill_table_n_byte[4]()


_, _ = bench[CRC32, CRC32_table_4_byte, rand_list, a_little_endian_table_4_byte]()

Function 1 runtime (ms): 	 10.85207628
Function 2 runtime (ms): 	 0.75994406349999999
Speedup factor: 		 14.280098761505755


A 14 times speedup over where we started!


But why stop there? 



## n-bytes implementation

We can in principle explicitly write out version that take 8, 16 or however many bytes at time. This get's a little long-winded, so I'll write some generic functions to make functions of arbitrary size. 


In [47]:
fn CRC32_table_n_byte_compact[size: Int](data: List[UInt8], table: List[UInt32]) -> UInt32:
    var crc32: UInt32 = 0xFFFFFFFF

    alias step_size = 4 # Always smashing 4 bytes into an UInt32
    alias units = size // step_size

    var length = len(data) // size
    var extra = len(data) % size

    var n = 0

    for i in range(start=0, end=length * size, step=size):
        @parameter
        for j in range(units):
            var vals = (
                (data[i + j * step_size + 3].cast[DType.uint32]() << 24)
                | (data[i + j * step_size + 2].cast[DType.uint32]() << 16)
                | (data[i + j * step_size + 1].cast[DType.uint32]() << 8)
                | (data[i + j * step_size + 0].cast[DType.uint32]() << 0)
            )

            if j == 0:
                vals = vals ^ crc32
                crc32 = 0 
        

            n = size - j * step_size
            crc32 = (
                table[(n - 4) * 256 + int((vals >> 24).cast[DType.uint8]())]
                ^ table[(n - 3) * 256 + int((vals >> 16).cast[DType.uint8]())]
                ^ table[(n - 2) * 256 + int((vals >> 8).cast[DType.uint8]())]
                ^ table[(n - 1) * 256 + int((vals >> 0).cast[DType.uint8]())]     
            ) ^ crc32
            

    for i in range(size * length, size * length + extra):
        var index = (crc32 ^ data[i].cast[DType.uint32]()) & 0xFF
        crc32 = table[int(index)] ^ (crc32 >> 8)

    return crc32 ^ 0xFFFFFFFF

In [48]:
print(hex(CRC32(example_list, List[UInt32](1))))
print(hex(CRC32_table_n_byte_compact[4](example_list, little_endian_table_4_byte)))


0x382aa34e
0x382aa34e


In [50]:
alias a_little_endian_table_4_byte = fill_table_n_byte[4]()

_, _ = bench[CRC32, CRC32_table_n_byte_compact[4], rand_list, a_little_endian_table_4_byte]()

Function 1 runtime (ms): 	 10.356216330000001
Function 2 runtime (ms): 	 0.78156012050000001
Speedup factor: 		 13.250696982050021


In [51]:

_, _ = bench[CRC32, CRC32_table_n_byte_compact[8], rand_list, fill_table_n_byte[8]()]()

Function 1 runtime (ms): 	 10.214365820000001
Function 2 runtime (ms): 	 0.56005104350000001
Speedup factor: 		 18.238276561661294


In [52]:
_, _ = bench[CRC32, CRC32_table_n_byte_compact[16], rand_list, fill_table_n_byte[16]()]()

Function 1 runtime (ms): 	 10.234594035000001
Function 2 runtime (ms): 	 0.43714066870483598
Speedup factor: 		 23.412587223520386


In [53]:
_, _ = bench[CRC32, CRC32_table_n_byte_compact[32], rand_list, fill_table_n_byte[32]()]()

Function 1 runtime (ms): 	 9.3334451200000004
Function 2 runtime (ms): 	 0.37502553692614765
Speedup factor: 		 24.887492186533422


In [54]:
_, _ = bench[CRC32, CRC32_table_n_byte_compact[64], rand_list, fill_table_n_byte[64]()]()

Function 1 runtime (ms): 	 8.5875961749999998
Function 2 runtime (ms): 	 0.39271820683813335
Speedup factor: 		 21.867069123534549


## A more realistic Python benchmark


Now as I hinted before, no-one in their right mind would write a CRC-32 check like I did above in Python. 

What one would really do is use `zlib` from Python's standard library. In this case we get: 

In [27]:
%%python

import zlib


py_rand_list = np.random.randint(0, 255, size=2**20, dtype=np.uint8)

realistic_python_seconds = timeit(lambda: zlib.crc32(py_rand_list), number=100)/100

print(f"Realistic Python runtime (ms): {realistic_python_seconds*100}")

Realistic Python runtime (ms): 0.08662398695014417


In [28]:
_, result_32_byte = bench[CRC32, CRC32_table_n_byte_compact[32], rand_list, fill_table_n_byte[32]()]()

Function 1 runtime (ms): 	 10.337301555
Function 2 runtime (ms): 	 0.22026469453344011
Speedup factor: 		 46.931268657903836


In [29]:
print("Python is:", result_32_byte/(realistic_python_seconds*100), "faster")

Python is: 2.5427679132364562 faster


So when using Python as you realistically would, Python is actually still three times faster than our best effort so far!



In [30]:
fn CRC32_table_4_byte_cast(owned data: List[UInt8], table: List[UInt32]) -> UInt32:
    var crc32: UInt32 = 0xffffffff

    var extra = len(data) % 4
    var leftover = List[SIMD[DType.uint8, 1]](capacity = extra)
    for i in range(extra):
        leftover.append(data[-(i + 1)])

    var result_length = len(data)//4
    var ptr_to_int8 = data.steal_data() 
    var ptr_to_uint32 = ptr_to_int8.bitcast[UInt32]()

    var result = List[UInt32]()
    result.data = ptr_to_uint32
    result.capacity = result_length
    result.size = result_length

    for word in result:
        var index = (crc32 ^ word[]) #& 0xff
        crc32 =  table[int((index >> 8).cast[DType.uint8]())] ^ table[256 + int(index.cast[DType.uint8]())] ^ (crc32 >> 16)
    
    for byte in leftover:
        var index = (crc32 ^ byte[].cast[DType.uint32]()) & 0xff
        crc32 = table[int(index)] ^ (crc32 >> 8)


    return crc32^0xffffffff

In [31]:
alias owned_data_table_func = fn(owned data: List[UInt8], table: List[UInt32]) -> UInt32


fn wrap_func_2[testing_function: owned_data_table_func, data: List[UInt8], table: List[UInt32] ]():
    var a =  testing_function(data = data, table = table)
    benchmark.keep(a)


fn bench_2[function_1: data_table_func, function_2: owned_data_table_func,
         test_list: List[UInt8], test_table: List[UInt32]]() raises -> Tuple[Float64, Float64]:


    var CRC32_1 = function_1(test_list, test_table) 

    alias function_1_wrapped = wrap_func[function_1]
    var report_1 = benchmark.run[function_1_wrapped[test_list, test_table]](max_runtime_secs=1.0
    ).mean(benchmark.Unit.ms)
    print("Function 1 runtime (ms): \t", report_1)


    var CRC32_2 = function_2(test_list, test_table) 
    
    #assert_equal(CRC32_1, CRC32_2)

    alias function_2_wrapped = wrap_func_2[function_2]
    var report_2 = benchmark.run[function_2_wrapped[test_list, test_table]](max_runtime_secs=1.0
    ).mean(benchmark.Unit.ms)
    print("Function 2 runtime (ms): \t", report_2)

    print("Speedup factor: \t\t", (report_1/report_2))

    return report_1, report_2

In [32]:
_, result_32_byte = bench_2[CRC32, CRC32_table_4_byte_cast, rand_list, fill_table_n_byte[4]()]()

Function 1 runtime (ms): 	 11.485327318840579
Function 2 runtime (ms): 	 0.73964751049999999
Speedup factor: 		 15.528109208501931


In [33]:
fn CRC32_table_braided(owned data: List[UInt8], table: List[UInt32]) -> UInt32:
    var crc32: UInt32 = 0xffffffff

    var extra = len(data) % 8
    var leftover = List[SIMD[DType.uint8, 1]](capacity = extra)
    for i in range(extra):
        leftover.append(data[-(i + 1)])

    var result_length = len(data)//8
    var ptr_to_int8 = data.steal_data() 
    var ptr_to_uint64 = ptr_to_int8.bitcast[UInt64]()

    var result = List[UInt64]()
    result.data = ptr_to_uint64
    result.capacity = result_length
    result.size = result_length

    for word in result:
        var index = (crc32 ^ word[]) #& 0xff
        crc32 =  table[int((index >> 8).cast[DType.uint8]())] ^ table[256 + int(index.cast[DType.uint8]())] ^ (crc32 >> 16)
    
    for byte in leftover:
        var index = (crc32 ^ byte[].cast[DType.uint32]()) & 0xff
        crc32 = table[int(index)] ^ (crc32 >> 8)


    return crc32^0xffffffff

error: Expression [33]:19:28: invalid call to '__xor__': callee expects at least 2 positional parameters, but 0 were specified
        var index = (crc32 ^ word[]) #& 0xff
                     ~~~~~~^~~~~~~~


Expression [0] wrapper:17:5: function declared here
    var __lldb_repl_python__ = __mojo_repl_Python()
    ^

expression failed to parse (no further compiler diagnostics)

In [ ]:
var crc = 0xedb88320
crc = (~crc) & 0xffffffff
print(hex(crc))

0x12477cdf


In [ ]:


fn multmodp(a: UInt64, owned b: UInt64, poly: UInt64) -> UInt64:
    """
    Return a(x) multiplied by b(x) modulo p(x), where p(x) is the CRC polynomial,
    reflected. For speed, this requires that a not be zero.
    """

    var m: UInt64 = 1 << 31
    var product: UInt64 = 0
    print("Enter multmodp")
    print(a)
    print(b)
    var cond = True
    while cond:
        if (a & m):
            product ^= b
            if ((a & (m - 1)) == 0):
                print("Breaks")
                cond = False
                break
        
        m >>= 1
        if  b & 1:
            b = (b >> 1) ^ poly
        else:
            b = b >> 1
    
    return product


In [ ]:
fn x2nmodp(owned n: UInt64, owned k: UInt64, x2n_table: List[UInt64], poly: UInt64) -> UInt64: 
    var product: UInt64 = 0

    product = 1 << 31       # x^0 == 1 
    

    while (n > 0):
        if (n & 1):
            product = multmodp(x2n_table[int(k & 31)], product, poly)
        n >>= 1
        k += 1
    
    return product

In [ ]:
from tensor import Tensor, TensorSpec, TensorShape



fn braid( n: Int , w: Int, poly: UInt64, x2n_table: List[UInt64]) -> Tensor[DType.uint64]:
    var spec = TensorSpec(DType.uint8, w, 256)
    var little_endian_table = Tensor[DType.uint64](spec)
    
    #var  = List[List[UInt64]]()
    var p: UInt64 = 0
    for k in range(w):
        p = x2nmodp(( n * w + 3 - k) << 3, 0, x2n_table, poly)
        little_endian_table[k][0] = 0
        for i in range(256):
            little_endian_table[k][i] = multmodp(i << 24, p, poly)

    return little_endian_table


In [ ]:
fn make_braided_crc_table[n: Int, poly: UInt64 = 0xEDB88320]() -> List[UInt32]:

    var table = List[UInt32](capacity=256 * n)
    table.size = 256*n

    var w: Int = 8
    # Initializa the first table as usual
    for i in range(256):

        var crc32 = UInt64(i)#.cast[DType.uint32]()
        for _ in range(8):
            if crc32 & 1 != 0:
                crc32 = (crc32 >> 1) ^ poly
            else:
                crc32 = crc32 >> 1

            table[i] = crc32

    # initialize the x^2^n mod p(x) table
    var x2n_table = List[UInt64](32)
    x2n_table.resize(32, 0)

    var p: UInt64 = (1 << 30)
    x2n_table[0] = p
    
    print("Initiate x2n_table")
    for entry in range(1, 32):
        x2n_table[entry] = 1 #multmodp(p, p, poly)

    # initialize braided table
    print("Initiate crc_braid_table")
    var crc_braid_table = braid(n, w, poly, x2n_table)


    return table

In [ ]:
var a = make_braided_crc_table[6]()

Initiate x2n_table
Initiate crc_braid_table
Enter multmodp
1
2147483648
Breaks
Enter multmodp
1
1
Breaks
Enter multmodp
1
2852767883
Breaks
Enter multmodp
1
2219446010
Breaks
Enter multmodp
0
101480190


error: Execution was interrupted, reason: signal SIGSTOP.
The process has been left at the point where it was interrupted, use "thread return -x" to return to the state before expression evaluation.


In [ ]:
print(hex(little_endian_table[1]))

0x77073096


In [ ]:
        else:
            var crc32 = table[i - 256]
            var index = int(crc32.cast[DType.uint8]())
            table[i] = (crc32 >> 8) ^ table[index]

In [ ]:
from random import rand

fn run_32[data: List[SIMD[DType.uint8, 1]] ]():
    var a =  CRC32(data)
    benchmark.keep(a)


fn run_32_inv[data: List[SIMD[DType.uint8, 1]] ]():
    var a = CRC32_inv(data)
    benchmark.keep(a)


fn run_32_table[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table(data, table)
    benchmark.keep(a)


fn fill_table() -> List[UInt32]:

    var table = List[UInt32](capacity=256)

    for i in range(256):

        var key = UInt8(i)
        var crc32 = key.cast[DType.uint32]()
        for i in range(8):
            if crc32 & 1 != 0:
                crc32 = (crc32 >> 1) ^ 0xedb88320
            else:
                crc32 = crc32 >> 1

        table[i] = crc32
    return table

fn bench():

    
    alias fill_size = 2**16
    alias g = UnsafePointer[SIMD[DType.uint8, 1]].alloc(fill_size)
    rand[DType.uint8](ptr =  g, size = fill_size)


    alias rand_list = List[SIMD[DType.uint8,1]](data = g, size = fill_size, capacity = fill_size)


    print(len(rand_list))

    var report = benchmark.run[run_32[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report)

    var report_2 = benchmark.run[run_32_inv[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_2)

 
    
    alias little_endian_table = fill_table()

    var report_3 = benchmark.run[run_32_table[rand_list, little_endian_table]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_3)

    print(100 * (report/report_2 -1))
    print(100 * (report_2/report_3 -1))
    print(100 * (report/report_3 -1))
    #report.print_full()
    #report_2.print_full()


bench()

error: Expression [33]:4:19: invalid call to 'CRC32': missing 1 required positional argument: 'dummy_table'
    var a =  CRC32(data)
             ~~~~~^~~~~~
error: Expression [33]:9:22: invalid call to 'CRC32_inv': missing 1 required positional argument: 'dummy_table'
    var a = CRC32_inv(data)
            ~~~~~~~~~^~~~~~
error: Expression [33]:43:48: no matching function in initialization
    alias rand_list = List[SIMD[DType.uint8,1]](data = g, size = fill_size, capacity = fill_size)
                      ~~~~~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Expression [1]:3:4: function declared here
fn CRC32(data: List[UInt8], dummy_table: List[UInt32]) -> UInt32:
   ^

Expression [2]:1:4: function declared here
fn CRC32_inv(data: List[UInt8], dummy_table: List[UInt32]) -> UInt32:
   ^

Expression [0] wrapper:17:5: candidate not viable: unknown keyword arguments: 'data', 'size', 'capacity'
    var __lldb_repl_python__ = __mojo_repl_Python()
    ^

Expression [0] wrapper:17:5: candidate not viable: unknown keyword arguments: 'data', 'size', 'capacity'
    var __lldb_repl_python__ = __mojo_repl_Python()
    ^

Expression [0] wrapper:17:5: candidate not viable: unknown keyword arguments: 'data', 'size'
    var __lldb_repl_python__ = __mojo_repl_Python()
    ^

Expression [0] wrapper:17:5: candidate not viable: unknown keyword arguments: 'data', 'size', 'capacity'
    var __lldb_repl_python__ = __mojo_repl_Python()
    ^

Expression [0] wrapper:17:5: candidate not viable: unknown keyword arguments: 'data', 'size', 'capacity'
    var __lldb_re

In [ ]:
var little_endian_table_2_byte = List[UInt32](capacity=512)

for i in range(256):

    var key = UInt8(i)
    var crc32 = key.cast[DType.uint32]()
    for i in range(8):
        if crc32 & 1 != 0:
            crc32 = (crc32 >> 1) ^ 0xedb88320
        else:
            crc32 = crc32 >> 1

    little_endian_table_2_byte[i] = crc32

for i in range(256, 512):
    var crc32 = little_endian_table_2_byte[i-256]
    little_endian_table_2_byte[i] = (crc32 >> 8) ^ little_endian_table_2_byte[int(crc32.cast[DType.uint8]())]






In [ ]:
fn CRC32_table_2_byte(owned data: List[UInt8], table: List[UInt32]) -> UInt32:
    var crc32: UInt32 = 0xffffffff

    var extra = len(data) % 2
    var leftover = List[SIMD[DType.uint8, 1]](capacity = extra)
    for i in range(extra):
        leftover.append(data[-(i + 1)])

    var result_length = len(data)//2
    var ptr_to_int8 = data.steal_data() 
    var ptr_to_uint16 = ptr_to_int8.bitcast[UInt16]()

    var result = List[UInt16]()
    result.data = ptr_to_uint16
    result.capacity = result_length
    result.size = result_length

    for byte in result:
        var index = (crc32 ^ byte[].cast[DType.uint32]()) #& 0xff
        crc32 =  table[int((index >> 8).cast[DType.uint8]())] ^ table[256 + int(index.cast[DType.uint8]())] ^ (crc32 >> 16)
    
    for byte in leftover:
        var index = (crc32 ^ byte[].cast[DType.uint32]()) & 0xff
        crc32 = table[int(index)] ^ (crc32 >> 8)


    return crc32^0xffffffff

In [ ]:
from testing import assert_true


fn CRC32_table_2_byte(owned data: List[SIMD[DType.uint8, 1]], table: List[UInt32]) -> SIMD[DType.uint32, 1]:
    var crc32: UInt32 = 0xffffffff

    #assert_true(len(data) % 2 == 0, "List must be divisible by two for 16-bit optimization.")

    var extra = len(data) % 2
    var leftover = List[SIMD[DType.uint8, 1]](capacity = extra)
    for i in range(extra):
        leftover.append(data[-(i + 1)])

    var result_length = len(data)//2
    var ptr_to_int8 = data.steal_data() 
    var ptr_to_uint16 = ptr_to_int8.bitcast[UInt16]()

    var result = List[UInt16]()
    result.data = ptr_to_uint16
    result.capacity = result_length
    result.size = result_length

    for byte in result:
        var index = (crc32 ^ byte[].cast[DType.uint32]()) #& 0xff
        crc32 =  table[int((index >> 8).cast[DType.uint8]())] ^ table[256 + int(index.cast[DType.uint8]())] ^ (crc32 >> 16)
    
    for byte in leftover:
        var index = (crc32 ^ byte[].cast[DType.uint32]()) & 0xff
        crc32 = table[int(index)] ^ (crc32 >> 8)


    return crc32^0xffffffff

In [ ]:
print(hex(CRC32(test_list)))
print(hex(CRC32_inv(test_list)))
print(hex(CRC32_table_2_byte(test_list, little_endian_table_2_byte)))

0x382aa34e
0x382aa34e
0x382aa34e


In [ ]:
var f: UInt32 = (0xff << 8) | 0xff
print(hex(f))

0xffff


In [ ]:
fn run_32[data: List[SIMD[DType.uint8, 1]] ]():
    var a =  CRC32(data)
    benchmark.keep(a)


fn run_32_inv[data: List[SIMD[DType.uint8, 1]] ]():
    var a = CRC32_inv(data)
    benchmark.keep(a)


fn run_32_table[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table(data, table)
    benchmark.keep(a)


fn run_32_table_2_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_2_byte(data, table)
    benchmark.keep(a)


fn fill_table() -> List[UInt32]:

    var table = List[UInt32](capacity=256)

    for i in range(256):

        var key = UInt8(i)
        var crc32 = key.cast[DType.uint32]()
        for i in range(8):
            if crc32 & 1 != 0:
                crc32 = (crc32 >> 1) ^ 0xedb88320
            else:
                crc32 = crc32 >> 1

        table[i] = crc32
    return table

fn fill_table_2_byte() -> List[UInt32]:

    var table = List[UInt32](capacity=512)
    table.size = 512

    for i in range(256):

        var key = UInt8(i)
        var crc32 = key.cast[DType.uint32]()
        for i in range(8):
            if crc32 & 1 != 0:
                crc32 = (crc32 >> 1) ^ 0xedb88320
            else:
                crc32 = crc32 >> 1

        table[i] = crc32

    for i in range(256, 512):
        var crc32 = table[i-256]
        table[i] = (crc32 >> 8) ^ table[int(crc32.cast[DType.uint8]())]
    return table



fn bench():

    
    alias fill_size = 2**16
    alias g = UnsafePointer[SIMD[DType.uint8, 1]].alloc(fill_size)
    rand[DType.uint8](ptr =  g, size = fill_size)


    alias rand_list = List[SIMD[DType.uint8,1]](data = g, size = fill_size, capacity = fill_size)


    print(len(rand_list))

    var report = benchmark.run[run_32[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report)

    var report_2 = benchmark.run[run_32_inv[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_2)

 
    
    alias little_endian_table = fill_table()

    var report_3 = benchmark.run[run_32_table[rand_list, little_endian_table]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_3)

    alias little_endian_table_2_byte = fill_table_2_byte()

    var report_4 = benchmark.run[run_32_table_2_byte[rand_list, little_endian_table_2_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_4)

    print(100 * (report/report_2 -1))
    print(100 * (report_2/report_3 -1))
    print(100 * (report/report_3 -1))
    print(100 * (report/report_4 -1))
    #report.print_full()
    #report_2.print_full()


bench()

65536
0.86205289350000003
0.7265950175
0.16436725754418979
0.092023411149999998
18.642830288882362
342.05581352153206
424.46752861848955
836.77563429466511


In [ ]:
fn CRC32_table_2_byte_2(owned data: List[SIMD[DType.uint8, 1]], table: List[UInt32]) -> SIMD[DType.uint32, 1]:
    var crc32: UInt32 = 0xffffffff

    var length = len(data)//2
    var extra = len(data) % 2

    for i in range(start = 0, end = length *2 , step = 2):
        
        var val: UInt32 = ((data[i + 1].cast[DType.uint32]() << 8) | data[i].cast[DType.uint32]())
        var index = crc32 ^ val
        crc32 =  table[int((index >> 8).cast[DType.uint8]())] ^ table[256 + int(index.cast[DType.uint8]())] ^ (crc32 >> 16)
    

    for i in range(2*length, 2*length + extra ):
        var index = (crc32 ^ data[i].cast[DType.uint32]()) & 0xff
        crc32 = table[int(index)] ^ (crc32 >> 8)


    return crc32^0xffffffff

In [ ]:
print(hex(CRC32(test_list)))
print(hex(CRC32_inv(test_list)))
print(hex(CRC32_table_2_byte(test_list, little_endian_table_2_byte)))
print(hex(CRC32_table_2_byte_2(test_list, little_endian_table_2_byte)))

0x382aa34e
0x382aa34e
0x382aa34e
0x382aa34e


In [ ]:
fn run_32[data: List[SIMD[DType.uint8, 1]] ]():
    var a =  CRC32(data)
    benchmark.keep(a)


fn run_32_inv[data: List[SIMD[DType.uint8, 1]] ]():
    var a = CRC32_inv(data)
    benchmark.keep(a)


fn run_32_table[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table(data, table)
    benchmark.keep(a)


fn run_32_table_2_byte_2[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_2_byte_2(data, table)
    benchmark.keep(a)


fn fill_table() -> List[UInt32]:

    var table = List[UInt32](capacity=256)

    for i in range(256):

        var key = UInt8(i)
        var crc32 = key.cast[DType.uint32]()
        for i in range(8):
            if crc32 & 1 != 0:
                crc32 = (crc32 >> 1) ^ 0xedb88320
            else:
                crc32 = crc32 >> 1

        table[i] = crc32
    return table



fn bench():

    
    alias fill_size = 2**16
    alias g = UnsafePointer[SIMD[DType.uint8, 1]].alloc(fill_size)
    rand[DType.uint8](ptr =  g, size = fill_size)


    alias rand_list = List[SIMD[DType.uint8,1]](data = g, size = fill_size, capacity = fill_size)


    print(len(rand_list))

    var report = benchmark.run[run_32[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report)

    var report_2 = benchmark.run[run_32_inv[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_2)

 
    
    alias little_endian_table = fill_table()

    var report_3 = benchmark.run[run_32_table[rand_list, little_endian_table]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_3)

    alias little_endian_table_2_byte = fill_table_2_byte()

    var report_4 = benchmark.run[run_32_table_2_byte_2[rand_list, little_endian_table_2_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_4)


    var report_5 = benchmark.run[run_32_table_2_byte[rand_list, little_endian_table_2_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_5)

    print(100 * (report/report_2 -1))
    print(100 * (report_2/report_3 -1))
    print(100 * (report/report_3 -1))
    print(100 * (report/report_4 -1))
    print(100 * (report/report_5 -1))
    #report.print_full()
    #report_2.print_full()


bench()

65536
0.83095668599999994
0.68331465049999995
0.18584170618320112
0.11355653557158424
0.098940018650000003
21.60674228072914
267.68638457634177
347.13143409308253
631.75593268797638
739.85903513876076


In [ ]:
fn fill_table_n_byte[n: Int]() -> List[UInt32]:

    var table = List[UInt32](capacity=256*n)
    table.size = 256*n

    for i in range(256*n):

        if i < 256: 
            var key = UInt8(i)
            var crc32 = key.cast[DType.uint32]()
            for i in range(8):
                if crc32 & 1 != 0:
                    crc32 = (crc32 >> 1) ^ 0xedb88320
                else:
                    crc32 = crc32 >> 1

            table[i] = crc32
        else:
            var crc32 = table[i-256]
            var index = int(crc32.cast[DType.uint8]())
            table[i] = (crc32 >> 8) ^ table[index]
            
    return table

In [ ]:
#var t = fill_table_n_byte[1]()
alias t2 = fill_table_n_byte[2]()

In [ ]:
fn CRC32_table_4_byte(owned data: List[SIMD[DType.uint8, 1]], table: List[UInt32]) -> SIMD[DType.uint32, 1]:
    var crc32: UInt32 = 0xffffffff

    var size = 4

    #assert_true(len(data) % 2 == 0, "List must be divisible by two for 16-bit optimization.")
    var length = len(data)//size
    var extra = len(data) % size



    for i in range(start = 0, end = length*size, step = size):
        
        var val: UInt32 =  (data[i + 3].cast[DType.uint32]() << 24) | (data[i + 2].cast[DType.uint32]() << 16) | (data[i + 1].cast[DType.uint32]() << 8) | data[i].cast[DType.uint32]()
        var index = crc32 ^ val.cast[DType.uint32]()
        crc32 = table[0*256 + int((index >> 24).cast[DType.uint8]())] ^
                table[1*256 + int((index >> 16).cast[DType.uint8]())] ^
                table[2*256 + int((index >> 8).cast[DType.uint8]())] ^
                table[3*256 + int((index >> 0).cast[DType.uint8]())] 
    
    for i in range(size*length, size*length + extra ):
        var index = (crc32 ^ data[i].cast[DType.uint32]()) & 0xff
        crc32 = table[int(index)] ^ (crc32 >> 8)


    return crc32^0xffffffff

In [ ]:



var little_endian_table_4_byte  = fill_table_n_byte[4]()

In [ ]:
print(hex(CRC32(test_list)))
print(hex(CRC32_inv(test_list)))
print(hex(CRC32_table_2_byte(test_list, little_endian_table_2_byte)))
print(hex(CRC32_table_2_byte_2(test_list, little_endian_table_2_byte)))
print(hex(CRC32_table_4_byte(test_list, little_endian_table_4_byte)))

0x382aa34e
0x382aa34e
0x382aa34e
0x382aa34e
0x382aa34e


In [ ]:
fn run_32[data: List[SIMD[DType.uint8, 1]] ]():
    var a =  CRC32(data)
    benchmark.keep(a)


fn run_32_inv[data: List[SIMD[DType.uint8, 1]] ]():
    var a = CRC32_inv(data)
    benchmark.keep(a)


fn run_32_table[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table(data, table)
    benchmark.keep(a)


fn run_32_table_2_byte_2[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_2_byte_2(data, table)
    benchmark.keep(a)

fn run_32_table_4_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_4_byte(data, table)
    benchmark.keep(a)



fn bench():

    
    alias fill_size = 2**16
    alias g = UnsafePointer[SIMD[DType.uint8, 1]].alloc(fill_size)
    rand[DType.uint8](ptr =  g, size = fill_size)


    alias rand_list = List[SIMD[DType.uint8,1]](data = g, size = fill_size, capacity = fill_size)


    print(len(rand_list))

    var report = benchmark.run[run_32[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report)

    var report_2 = benchmark.run[run_32_inv[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_2)

 
    
    alias little_endian_table = fill_table_n_byte[1]()

    var report_3 = benchmark.run[run_32_table[rand_list, little_endian_table]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_3)

    alias little_endian_table_2_byte = fill_table_n_byte[2]()

    var report_4 = benchmark.run[run_32_table_2_byte_2[rand_list, little_endian_table_2_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_4)

    alias little_endian_table_4_byte = fill_table_n_byte[4]()

    var report_5 = benchmark.run[run_32_table_4_byte[rand_list, little_endian_table_4_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_5)

    print(100 * (report/report_2 -1))
    print(100 * (report_2/report_3 -1))
    print(100 * (report/report_3 -1))
    print(100 * (report/report_4 -1))
    print(100 * (report/report_5 -1))
    #report.print_full()
    #report_2.print_full()


bench()

65536
0.79050925449999998
0.5410031475
0.12362461018886081
0.083568155249999998
0.0433552434
46.119159962927945
337.6176771546634
539.44327370767212
845.9455604053195
1723.3302191540688


In [ ]:
fn CRC32_table_8_byte(owned data: List[SIMD[DType.uint8, 1]], table: List[UInt32]) -> SIMD[DType.uint32, 1]:
    var crc32: UInt32 = 0xffffffff

    var size = 8

    var length = len(data)//size
    var extra = len(data) % size



    for i in range(start = 0, end = length*size, step = size):
        
        var val_1: UInt32 = (data[i + 3].cast[DType.uint32]() << 24) | 
                            (data[i + 2].cast[DType.uint32]() << 16) | 
                            (data[i + 1].cast[DType.uint32]() << 8) | 
                             data[i + 0].cast[DType.uint32]()

        var val_2: UInt32 = (data[i + 7].cast[DType.uint32]() << 24) | 
                            (data[i + 6].cast[DType.uint32]() << 16) | 
                            (data[i + 5].cast[DType.uint32]() << 8) | 
                             data[i + 4].cast[DType.uint32]()

        var index_1 = crc32 ^ val_1#.cast[DType.uint32]()
        var index_2 = val_2#.cast[DType.uint32]()
        crc32 = table[4*256 + int((index_1 >> 24).cast[DType.uint8]())] ^
                table[5*256 + int((index_1 >> 16).cast[DType.uint8]())] ^
                table[6*256 + int((index_1 >> 8).cast[DType.uint8]())] ^
                table[7*256 + int((index_1 >> 0).cast[DType.uint8]())] ^
                table[0*256 + int((index_2 >> 24).cast[DType.uint8]())] ^
                table[1*256 + int((index_2 >> 16).cast[DType.uint8]())] ^
                table[2*256 + int((index_2 >> 8).cast[DType.uint8]())] ^
                table[3*256 + int((index_2 >> 0).cast[DType.uint8]())] 
    
    for i in range(size*length, size*length + extra ):
        var index = (crc32 ^ data[i].cast[DType.uint32]()) & 0xff
        crc32 = table[int(index)] ^ (crc32 >> 8)


    return crc32^0xffffffff

In [ ]:
fn CRC32_table_n_byte_compact[
    size: Int
](owned data: List[SIMD[DType.uint8, 1]], table: List[UInt32]) -> SIMD[DType.uint32, 1]:
    var crc32: UInt32 = 0xFFFFFFFF

    alias step_size = 4
    alias units = size // step_size

    var length = len(data) // size
    var extra = len(data) % size

    var vals = List[UInt32](capacity=units)
    vals.size = units
    var n = 0
    for i in range(start=0, end=length * size, step=size):


        @unroll(units)
        for j in range(units):
            vals[j] = (
                (data[i + j * step_size + 3].cast[DType.uint32]() << 24)
                | (data[i + j * step_size + 2].cast[DType.uint32]() << 16)
                | (data[i + j * step_size + 1].cast[DType.uint32]() << 8)
                | (data[i + j * step_size + 0].cast[DType.uint32]() << 0)
            )

            if j == 0:
                vals[0] = vals[0] ^ crc32
                crc32 = 0

            n = size - j * step_size
            crc32 = (
                table[(n - 4) * 256 + int((vals[j] >> 24).cast[DType.uint8]())]
                ^ table[(n - 3) * 256 + int((vals[j] >> 16).cast[DType.uint8]())]
                ^ table[(n - 2) * 256 + int((vals[j] >> 8).cast[DType.uint8]())]
                ^ table[(n - 1) * 256 + int((vals[j] >> 0).cast[DType.uint8]())]
                ^ crc32
            )
    for i in range(size * length, size * length + extra):
        var index = (crc32 ^ data[i].cast[DType.uint32]()) & 0xFF
        crc32 = table[int(index)] ^ (crc32 >> 8)

    return crc32 ^ 0xFFFFFFFF


In [ ]:
var little_endian_table_8_byte  = fill_table_n_byte[8]()
print(hex(CRC32(test_list)))
print(hex(CRC32_inv(test_list)))
print(hex(CRC32_table_8_byte(test_list, little_endian_table_8_byte)))
print(hex(CRC32_table_n_byte_compact[8](test_list, little_endian_table_8_byte)))

0x382aa34e
0x382aa34e
0x382aa34e
0x382aa34e


In [ ]:
fn run_32[data: List[SIMD[DType.uint8, 1]] ]():
    var a =  CRC32(data)
    benchmark.keep(a)


fn run_32_inv[data: List[SIMD[DType.uint8, 1]] ]():
    var a = CRC32_inv(data)
    benchmark.keep(a)


fn run_32_table[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table(data, table)
    benchmark.keep(a)


fn run_32_table_2_byte_2[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_2_byte_2(data, table)
    benchmark.keep(a)

fn run_32_table_4_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_4_byte(data, table)
    benchmark.keep(a)

fn run_32_table_8_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_8_byte(data, table)
    benchmark.keep(a)

fn run_32_table_8_byte_compact[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_n_byte_compact[8](data, table)
    benchmark.keep(a)


fn bench():

    
    alias fill_size = 2**20
    alias g = UnsafePointer[SIMD[DType.uint8, 1]].alloc(fill_size)
    rand[DType.uint8](ptr =  g, size = fill_size)


    alias rand_list = List[SIMD[DType.uint8,1]](data = g, size = fill_size, capacity = fill_size)


    print(len(rand_list))

    var report = benchmark.run[run_32[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report)

    var report_2 = benchmark.run[run_32_inv[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_2)

 
    
    alias little_endian_table = fill_table_n_byte[1]()

    var report_3 = benchmark.run[run_32_table[rand_list, little_endian_table]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_3)

    alias little_endian_table_2_byte = fill_table_n_byte[2]()

    var report_4 = benchmark.run[run_32_table_2_byte_2[rand_list, little_endian_table_2_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_4)

    alias little_endian_table_4_byte = fill_table_n_byte[4]()

    var report_5 = benchmark.run[run_32_table_4_byte[rand_list, little_endian_table_4_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_5)

    alias little_endian_table_8_byte = fill_table_n_byte[8]()




    var report_6 = benchmark.run[run_32_table_8_byte[rand_list, little_endian_table_8_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_6)

    var report_7 = benchmark.run[run_32_table_8_byte_compact[rand_list, little_endian_table_8_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_7)



    print(100 * (report/report_2 -1))
    print(100 * (report_2/report_3 -1))
    print(100 * (report/report_3 -1))
    print(100 * (report/report_4 -1))
    print(100 * (report/report_5 -1))
    print(100 * (report/report_6 -1))
    print(100 * (report/report_7 -1))

    #report.print_full()
    #report_2.print_full()


bench()

1048576
11.288160865671642
8.4133244950000012
2.082689491438356
1.669156014851485
0.77359028100000005
0.49301487150000001
0.49764872500000001
34.170040301906134
303.964418584047
441.99922321957672
576.2795547710391
1359.1911432857883
2189.618735298463
2168.2989624200568


In [ ]:
#var little_endian_table_8_byte  = fill_table_n_byte[8]()
#print(hex(CRC32(test_list)))
#print(hex(CRC32_inv(test_list)))
#print(hex(CRC32_table_8_byte_compact(test_list, little_endian_table_8_byte)))

In [ ]:
fn CRC32_table_16_byte(owned data: List[SIMD[DType.uint8, 1]], table: List[UInt32]) -> SIMD[DType.uint32, 1]:
    var crc32: UInt32 = 0xffffffff

    var size = 16

    var length = len(data)//size
    var extra = len(data) % size



    for i in range(start = 0, end = length*size, step = size):
        
        var val_1: UInt32 = (data[i + 3].cast[DType.uint32]() << 24) | 
                            (data[i + 2].cast[DType.uint32]() << 16) | 
                            (data[i + 1].cast[DType.uint32]() << 8) | 
                             data[i + 0].cast[DType.uint32]()

        var val_2: UInt32 = (data[i + 7].cast[DType.uint32]() << 24) | 
                            (data[i + 6].cast[DType.uint32]() << 16) | 
                            (data[i + 5].cast[DType.uint32]() << 8) | 
                             data[i + 4].cast[DType.uint32]()

        var val_3: UInt32 = (data[i + 11].cast[DType.uint32]() << 24) | 
                            (data[i + 10].cast[DType.uint32]() << 16) | 
                            (data[i + 9].cast[DType.uint32]() << 8) | 
                             data[i + 8].cast[DType.uint32]()

        var val_4: UInt32 = (data[i + 15].cast[DType.uint32]() << 24) | 
                            (data[i + 14].cast[DType.uint32]() << 16) | 
                            (data[i + 13].cast[DType.uint32]() << 8) | 
                             data[i + 12].cast[DType.uint32]()

        var index_1 = crc32 ^ val_1.cast[DType.uint32]()
        var index_2 = val_2.cast[DType.uint32]()
        var index_3 = val_3.cast[DType.uint32]()
        var index_4 = val_4.cast[DType.uint32]()

        crc32 = table[0*256 + int((index_4 >> 24).cast[DType.uint8]())] ^
                table[1*256 + int((index_4 >> 16).cast[DType.uint8]())] ^
                table[2*256 + int((index_4 >> 8).cast[DType.uint8]())] ^
                table[3*256 + int((index_4 >> 0).cast[DType.uint8]())] ^
                table[4*256 + int((index_3 >> 24).cast[DType.uint8]())] ^
                table[5*256 + int((index_3 >> 16).cast[DType.uint8]())] ^
                table[6*256 + int((index_3 >> 8).cast[DType.uint8]())] ^
                table[7*256 + int((index_3 >> 0).cast[DType.uint8]())] ^
                table[8*256 + int((index_2 >> 24).cast[DType.uint8]())] ^
                table[9*256 + int((index_2 >> 16).cast[DType.uint8]())] ^
                table[10*256 + int((index_2 >> 8).cast[DType.uint8]())] ^
                table[11*256 + int((index_2 >> 0).cast[DType.uint8]())] ^
                table[12*256 + int((index_1 >> 24).cast[DType.uint8]())] ^
                table[13*256 + int((index_1 >> 16).cast[DType.uint8]())] ^
                table[14*256 + int((index_1 >> 8).cast[DType.uint8]())] ^
                table[15*256 + int((index_1 >> 0).cast[DType.uint8]())] 
    
    for i in range(size*length, size*length + extra ):
        var index = (crc32 ^ data[i].cast[DType.uint32]()) & 0xff
        crc32 = table[int(index)] ^ (crc32 >> 8)


    return crc32^0xffffffff

In [ ]:
var little_endian_table_16_byte  = fill_table_n_byte[16]()
print(hex(CRC32(test_list)))
print(hex(CRC32_inv(test_list)))
print(hex(CRC32_table_16_byte(test_list, little_endian_table_16_byte)))

0x382aa34e
0x382aa34e
0x382aa34e


In [ ]:
fn run_32[data: List[SIMD[DType.uint8, 1]] ]():
    var a =  CRC32(data)
    benchmark.keep(a)


fn run_32_inv[data: List[SIMD[DType.uint8, 1]] ]():
    var a = CRC32_inv(data)
    benchmark.keep(a)


fn run_32_table[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table(data, table)
    benchmark.keep(a)


fn run_32_table_2_byte_2[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_2_byte_2(data, table)
    benchmark.keep(a)

fn run_32_table_4_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_4_byte(data, table)
    benchmark.keep(a)

fn run_32_table_8_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_8_byte(data, table)
    benchmark.keep(a)


fn run_32_table_16_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_16_byte(data, table)
    benchmark.keep(a)

fn bench():

    
    alias fill_size = 2**20
    alias g = UnsafePointer[SIMD[DType.uint8, 1]].alloc(fill_size)
    rand[DType.uint8](ptr =  g, size = fill_size)


    alias rand_list = List[SIMD[DType.uint8,1]](data = g, size = fill_size, capacity = fill_size)


    print(len(rand_list))

    var report = benchmark.run[run_32[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report)

    var report_2 = benchmark.run[run_32_inv[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_2)

 
    
    alias little_endian_table = fill_table_n_byte[1]()

    var report_3 = benchmark.run[run_32_table[rand_list, little_endian_table]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_3)

    alias little_endian_table_2_byte = fill_table_n_byte[2]()

    var report_4 = benchmark.run[run_32_table_2_byte_2[rand_list, little_endian_table_2_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_4)

    alias little_endian_table_4_byte = fill_table_n_byte[4]()

    var report_5 = benchmark.run[run_32_table_4_byte[rand_list, little_endian_table_4_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_5)

    alias little_endian_table_8_byte = fill_table_n_byte[8]()

    var report_6 = benchmark.run[run_32_table_8_byte[rand_list, little_endian_table_8_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_6)

    alias little_endian_table_16_byte = fill_table_n_byte[16]()

    var report_7 = benchmark.run[run_32_table_16_byte[rand_list, little_endian_table_16_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_7)

    print(100 * (report/report_2 -1))
    print(100 * (report_2/report_3 -1))
    print(100 * (report/report_3 -1))
    print(100 * (report/report_4 -1))
    print(100 * (report/report_5 -1))
    print(100 * (report/report_6 -1))
    print(100 * (report/report_7 -1))


    #report.print_full()
    #report_2.print_full()


bench()

1048576
10.729803022222223
8.2542810899999992
1.9779388177299086
1.3382771795160382
0.71156432250000001
0.47510569199999997
0.236569169
29.990763644108263
317.3173111326812
442.47395956043158
701.76238423959717
1407.9175111709205
2158.4033832670275
4435.5880766619348


In [ ]:
fn CRC32_table_32_byte(owned data: List[SIMD[DType.uint8, 1]], table: List[UInt32]) -> SIMD[DType.uint32, 1]:
    var crc32: UInt32 = 0xffffffff

    var size = 32

    var length = len(data)//size
    var extra = len(data) % size



    for i in range(start = 0, end = length*size, step = size):
        
        var val_1: UInt32 = (data[i + 3].cast[DType.uint32]() << 24) | 
                            (data[i + 2].cast[DType.uint32]() << 16) | 
                            (data[i + 1].cast[DType.uint32]() << 8) | 
                             data[i + 0].cast[DType.uint32]()

        var val_2: UInt32 = (data[i + 7].cast[DType.uint32]() << 24) | 
                            (data[i + 6].cast[DType.uint32]() << 16) | 
                            (data[i + 5].cast[DType.uint32]() << 8) | 
                             data[i + 4].cast[DType.uint32]()

        var val_3: UInt32 = (data[i + 11].cast[DType.uint32]() << 24) | 
                            (data[i + 10].cast[DType.uint32]() << 16) | 
                            (data[i + 9].cast[DType.uint32]() << 8) | 
                             data[i + 8].cast[DType.uint32]()

        var val_4: UInt32 = (data[i + 15].cast[DType.uint32]() << 24) | 
                            (data[i + 14].cast[DType.uint32]() << 16) | 
                            (data[i + 13].cast[DType.uint32]() << 8) | 
                             data[i + 12].cast[DType.uint32]()

        var val_5: UInt32 = (data[i + 19].cast[DType.uint32]() << 24) | 
                            (data[i + 18].cast[DType.uint32]() << 16) | 
                            (data[i + 17].cast[DType.uint32]() << 8) | 
                             data[i + 16].cast[DType.uint32]()

        var val_6: UInt32 = (data[i + 23].cast[DType.uint32]() << 24) | 
                            (data[i + 22].cast[DType.uint32]() << 16) | 
                            (data[i + 21].cast[DType.uint32]() << 8) | 
                             data[i + 20].cast[DType.uint32]()

        var val_7: UInt32 = (data[i + 27].cast[DType.uint32]() << 24) | 
                            (data[i + 26].cast[DType.uint32]() << 16) | 
                            (data[i + 25].cast[DType.uint32]() << 8) | 
                             data[i + 24].cast[DType.uint32]()

        var val_8: UInt32 = (data[i + 31].cast[DType.uint32]() << 24) | 
                            (data[i + 30].cast[DType.uint32]() << 16) | 
                            (data[i + 29].cast[DType.uint32]() << 8) | 
                             data[i + 28].cast[DType.uint32]()

        var index_1 = crc32 ^ val_1.cast[DType.uint32]()
        var index_2 = val_2.cast[DType.uint32]()
        var index_3 = val_3.cast[DType.uint32]()
        var index_4 = val_4.cast[DType.uint32]()
        var index_5 = val_5.cast[DType.uint32]()
        var index_6 = val_6.cast[DType.uint32]()
        var index_7 = val_7.cast[DType.uint32]()
        var index_8 = val_8.cast[DType.uint32]()

        crc32 = table[0*256 + int((index_8 >> 24).cast[DType.uint8]())] ^
                table[1*256 + int((index_8 >> 16).cast[DType.uint8]())] ^
                table[2*256 + int((index_8 >> 8).cast[DType.uint8]())] ^
                table[3*256 + int((index_8 >> 0).cast[DType.uint8]())] ^
                table[4*256 + int((index_7 >> 24).cast[DType.uint8]())] ^
                table[5*256 + int((index_7 >> 16).cast[DType.uint8]())] ^
                table[6*256 + int((index_7 >> 8).cast[DType.uint8]())] ^
                table[7*256 + int((index_7 >> 0).cast[DType.uint8]())] ^
                table[8*256 + int((index_6 >> 24).cast[DType.uint8]())] ^
                table[9*256 + int((index_6 >> 16).cast[DType.uint8]())] ^
                table[10*256 + int((index_6 >> 8).cast[DType.uint8]())] ^
                table[11*256 + int((index_6 >> 0).cast[DType.uint8]())] ^
                table[12*256 + int((index_5 >> 24).cast[DType.uint8]())] ^
                table[13*256 + int((index_5 >> 16).cast[DType.uint8]())] ^
                table[14*256 + int((index_5 >> 8).cast[DType.uint8]())] ^
                table[15*256 + int((index_5 >> 0).cast[DType.uint8]())] ^
                table[16*256 + int((index_4 >> 24).cast[DType.uint8]())] ^
                table[17*256 + int((index_4 >> 16).cast[DType.uint8]())] ^
                table[18*256 + int((index_4 >> 8).cast[DType.uint8]())] ^
                table[19*256 + int((index_4 >> 0).cast[DType.uint8]())] ^
                table[20*256 + int((index_3 >> 24).cast[DType.uint8]())] ^
                table[21*256 + int((index_3 >> 16).cast[DType.uint8]())] ^
                table[22*256 + int((index_3 >> 8).cast[DType.uint8]())] ^
                table[23*256 + int((index_3 >> 0).cast[DType.uint8]())] ^
                table[24*256 + int((index_2 >> 24).cast[DType.uint8]())] ^
                table[25*256 + int((index_2 >> 16).cast[DType.uint8]())] ^
                table[26*256 + int((index_2 >> 8).cast[DType.uint8]())] ^
                table[27*256 + int((index_2 >> 0).cast[DType.uint8]())] ^
                table[28*256 + int((index_1 >> 24).cast[DType.uint8]())] ^
                table[29*256 + int((index_1 >> 16).cast[DType.uint8]())] ^
                table[30*256 + int((index_1 >> 8).cast[DType.uint8]())] ^
                table[31*256 + int((index_1 >> 0).cast[DType.uint8]())] 
    
    for i in range(size*length, size*length + extra ):
        var index = (crc32 ^ data[i].cast[DType.uint32]()) & 0xff
        crc32 = table[int(index)] ^ (crc32 >> 8)


    return crc32^0xffffffff

In [ ]:
var little_endian_table_32_byte  = fill_table_n_byte[32]()
print(hex(CRC32(test_list)))
print(hex(CRC32_inv(test_list)))
print(hex(CRC32_table_32_byte(test_list, little_endian_table_32_byte)))

0x382aa34e
0x382aa34e
0x382aa34e


In [ ]:
fn run_32[data: List[SIMD[DType.uint8, 1]] ]():
    var a =  CRC32(data)
    benchmark.keep(a)


fn run_32_inv[data: List[SIMD[DType.uint8, 1]] ]():
    var a = CRC32_inv(data)
    benchmark.keep(a)


fn run_32_table[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table(data, table)
    benchmark.keep(a)


fn run_32_table_2_byte_2[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_2_byte_2(data, table)
    benchmark.keep(a)

fn run_32_table_4_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_4_byte(data, table)
    benchmark.keep(a)

fn run_32_table_8_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_8_byte(data, table)
    benchmark.keep(a)


fn run_32_table_16_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_16_byte(data, table)
    benchmark.keep(a)

fn run_32_table_32_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_32_byte(data, table)
    benchmark.keep(a)

fn bench():

    
    alias fill_size = 2**20
    alias g = UnsafePointer[SIMD[DType.uint8, 1]].alloc(fill_size)
    rand[DType.uint8](ptr =  g, size = fill_size)


    alias rand_list = List[SIMD[DType.uint8,1]](data = g, size = fill_size, capacity = fill_size)


    print(len(rand_list))

    var report = benchmark.run[run_32[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report)

    var report_2 = benchmark.run[run_32_inv[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_2)

 
    
    alias little_endian_table = fill_table_n_byte[1]()

    var report_3 = benchmark.run[run_32_table[rand_list, little_endian_table]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_3)

    alias little_endian_table_2_byte = fill_table_n_byte[2]()

    var report_4 = benchmark.run[run_32_table_2_byte_2[rand_list, little_endian_table_2_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_4)

    alias little_endian_table_4_byte = fill_table_n_byte[4]()

    var report_5 = benchmark.run[run_32_table_4_byte[rand_list, little_endian_table_4_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_5)

    alias little_endian_table_8_byte = fill_table_n_byte[8]()

    var report_6 = benchmark.run[run_32_table_8_byte[rand_list, little_endian_table_8_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_6)

    alias little_endian_table_16_byte = fill_table_n_byte[16]()

    var report_7 = benchmark.run[run_32_table_16_byte[rand_list, little_endian_table_16_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_7)

    alias little_endian_table_32_byte = fill_table_n_byte[32]()

    var report_8 = benchmark.run[run_32_table_32_byte[rand_list, little_endian_table_32_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_8)

    print(100 * (report/report_2 -1))
    print(100 * (report_2/report_3 -1))
    print(100 * (report/report_3 -1))
    print(100 * (report/report_4 -1))
    print(100 * (report/report_5 -1))
    print(100 * (report/report_6 -1))
    print(100 * (report/report_7 -1))
    print(100 * (report/report_8 -1))



    #report.print_full()
    #report_2.print_full()


bench()

1048576
8.6481844649999999
6.8747104100000005
1.9391274461538461
1.3276128694677871
0.7069171955000001
0.48138664549999999
0.2292578575
0.19133430500078627
25.79707288353983
254.52597113385275
345.98329429833007
551.40860441243535
1123.3659783708003
1696.5152431716137
3672.2521527969875
4419.9340834172217


In [ ]:
fn CRC32_table_64_byte(owned data: List[SIMD[DType.uint8, 1]], table: List[UInt32]) -> SIMD[DType.uint32, 1]:
    var crc32: UInt32 = 0xffffffff

    var size = 64

    var length = len(data)//size
    var extra = len(data) % size



    for i in range(start = 0, end = length*size, step = size):
        
        var val_1: UInt32 = (data[i + 3].cast[DType.uint32]() << 24) | 
                            (data[i + 2].cast[DType.uint32]() << 16) | 
                            (data[i + 1].cast[DType.uint32]() << 8) | 
                             data[i + 0].cast[DType.uint32]()

        var val_2: UInt32 = (data[i + 7].cast[DType.uint32]() << 24) | 
                            (data[i + 6].cast[DType.uint32]() << 16) | 
                            (data[i + 5].cast[DType.uint32]() << 8) | 
                             data[i + 4].cast[DType.uint32]()

        var val_3: UInt32 = (data[i + 11].cast[DType.uint32]() << 24) | 
                            (data[i + 10].cast[DType.uint32]() << 16) | 
                            (data[i + 9].cast[DType.uint32]() << 8) | 
                             data[i + 8].cast[DType.uint32]()

        var val_4: UInt32 = (data[i + 15].cast[DType.uint32]() << 24) | 
                            (data[i + 14].cast[DType.uint32]() << 16) | 
                            (data[i + 13].cast[DType.uint32]() << 8) | 
                             data[i + 12].cast[DType.uint32]()

        var val_5: UInt32 = (data[i + 19].cast[DType.uint32]() << 24) | 
                            (data[i + 18].cast[DType.uint32]() << 16) | 
                            (data[i + 17].cast[DType.uint32]() << 8) | 
                             data[i + 16].cast[DType.uint32]()

        var val_6: UInt32 = (data[i + 23].cast[DType.uint32]() << 24) | 
                            (data[i + 22].cast[DType.uint32]() << 16) | 
                            (data[i + 21].cast[DType.uint32]() << 8) | 
                             data[i + 20].cast[DType.uint32]()

        var val_7: UInt32 = (data[i + 27].cast[DType.uint32]() << 24) | 
                            (data[i + 26].cast[DType.uint32]() << 16) | 
                            (data[i + 25].cast[DType.uint32]() << 8) | 
                             data[i + 24].cast[DType.uint32]()

        var val_8: UInt32 = (data[i + 31].cast[DType.uint32]() << 24) | 
                            (data[i + 30].cast[DType.uint32]() << 16) | 
                            (data[i + 29].cast[DType.uint32]() << 8) | 
                             data[i + 28].cast[DType.uint32]()


        var val_9: UInt32 = (data[i + 35].cast[DType.uint32]() << 24) | 
                            (data[i + 34].cast[DType.uint32]() << 16) | 
                            (data[i + 33].cast[DType.uint32]() << 8) | 
                             data[i + 32].cast[DType.uint32]()

        var val_10: UInt32 =(data[i + 39].cast[DType.uint32]() << 24) | 
                            (data[i + 38].cast[DType.uint32]() << 16) | 
                            (data[i + 37].cast[DType.uint32]() << 8) | 
                             data[i + 36].cast[DType.uint32]()

        var val_11: UInt32 =(data[i + 43].cast[DType.uint32]() << 24) | 
                            (data[i + 42].cast[DType.uint32]() << 16) | 
                            (data[i + 41].cast[DType.uint32]() << 8) | 
                             data[i + 40].cast[DType.uint32]()

        var val_12: UInt32 =(data[i + 47].cast[DType.uint32]() << 24) | 
                            (data[i + 46].cast[DType.uint32]() << 16) | 
                            (data[i + 45].cast[DType.uint32]() << 8) | 
                             data[i + 44].cast[DType.uint32]()

        var val_13: UInt32 =(data[i + 51].cast[DType.uint32]() << 24) | 
                            (data[i + 50].cast[DType.uint32]() << 16) | 
                            (data[i + 49].cast[DType.uint32]() << 8) | 
                             data[i + 48].cast[DType.uint32]()

        var val_14: UInt32 =(data[i + 55].cast[DType.uint32]() << 24) | 
                            (data[i + 54].cast[DType.uint32]() << 16) | 
                            (data[i + 53].cast[DType.uint32]() << 8) | 
                             data[i + 52].cast[DType.uint32]()

        var val_15: UInt32 =(data[i + 59].cast[DType.uint32]() << 24) | 
                            (data[i + 58].cast[DType.uint32]() << 16) | 
                            (data[i + 57].cast[DType.uint32]() << 8) | 
                             data[i + 56].cast[DType.uint32]()

        var val_16: UInt32 =(data[i + 63].cast[DType.uint32]() << 24) | 
                            (data[i + 62].cast[DType.uint32]() << 16) | 
                            (data[i + 61].cast[DType.uint32]() << 8) | 
                             data[i + 60].cast[DType.uint32]()

        var index_1 = crc32 ^ val_1.cast[DType.uint32]()
        var index_2 = val_2.cast[DType.uint32]()
        var index_3 = val_3.cast[DType.uint32]()
        var index_4 = val_4.cast[DType.uint32]()
        var index_5 = val_5.cast[DType.uint32]()
        var index_6 = val_6.cast[DType.uint32]()
        var index_7 = val_7.cast[DType.uint32]()
        var index_8 = val_8.cast[DType.uint32]()
        var index_9 = val_9.cast[DType.uint32]()
        var index_10 = val_10.cast[DType.uint32]()
        var index_11 = val_11.cast[DType.uint32]()
        var index_12 = val_12.cast[DType.uint32]()
        var index_13 = val_13.cast[DType.uint32]()
        var index_14 = val_14.cast[DType.uint32]()
        var index_15 = val_15.cast[DType.uint32]()
        var index_16 = val_16.cast[DType.uint32]()

        crc32 = table[0*256 + int((index_16 >> 24).cast[DType.uint8]())] ^
                table[1*256 + int((index_16 >> 16).cast[DType.uint8]())] ^
                table[2*256 + int((index_16 >> 8).cast[DType.uint8]())] ^
                table[3*256 + int((index_16 >> 0).cast[DType.uint8]())] ^
                table[4*256 + int((index_15 >> 24).cast[DType.uint8]())] ^
                table[5*256 + int((index_15 >> 16).cast[DType.uint8]())] ^
                table[6*256 + int((index_15 >> 8).cast[DType.uint8]())] ^
                table[7*256 + int((index_15 >> 0).cast[DType.uint8]())] ^
                table[8*256 + int((index_14 >> 24).cast[DType.uint8]())] ^
                table[9*256 + int((index_14 >> 16).cast[DType.uint8]())] ^
                table[10*256 + int((index_14 >> 8).cast[DType.uint8]())] ^
                table[11*256 + int((index_14 >> 0).cast[DType.uint8]())] ^
                table[12*256 + int((index_13 >> 24).cast[DType.uint8]())] ^
                table[13*256 + int((index_13 >> 16).cast[DType.uint8]())] ^
                table[14*256 + int((index_13 >> 8).cast[DType.uint8]())] ^
                table[15*256 + int((index_13 >> 0).cast[DType.uint8]())] ^
                table[16*256 + int((index_12 >> 24).cast[DType.uint8]())] ^
                table[17*256 + int((index_12 >> 16).cast[DType.uint8]())] ^
                table[18*256 + int((index_12 >> 8).cast[DType.uint8]())] ^
                table[19*256 + int((index_12 >> 0).cast[DType.uint8]())] ^
                table[20*256 + int((index_11 >> 24).cast[DType.uint8]())] ^
                table[21*256 + int((index_11 >> 16).cast[DType.uint8]())] ^
                table[22*256 + int((index_11 >> 8).cast[DType.uint8]())] ^
                table[23*256 + int((index_11 >> 0).cast[DType.uint8]())] ^
                table[24*256 + int((index_10 >> 24).cast[DType.uint8]())] ^
                table[25*256 + int((index_10 >> 16).cast[DType.uint8]())] ^
                table[26*256 + int((index_10 >> 8).cast[DType.uint8]())] ^
                table[27*256 + int((index_10 >> 0).cast[DType.uint8]())] ^
                table[28*256 + int((index_9 >> 24).cast[DType.uint8]())] ^
                table[29*256 + int((index_9 >> 16).cast[DType.uint8]())] ^
                table[30*256 + int((index_9 >> 8).cast[DType.uint8]())] ^
                table[31*256 + int((index_9 >> 0).cast[DType.uint8]())] ^
                table[32*256 + int((index_8 >> 24).cast[DType.uint8]())] ^
                table[33*256 + int((index_8 >> 16).cast[DType.uint8]())] ^
                table[34*256 + int((index_8 >> 8).cast[DType.uint8]())] ^
                table[35*256 + int((index_8 >> 0).cast[DType.uint8]())] ^
                table[36*256 + int((index_7 >> 24).cast[DType.uint8]())] ^
                table[37*256 + int((index_7 >> 16).cast[DType.uint8]())] ^
                table[38*256 + int((index_7 >> 8).cast[DType.uint8]())] ^
                table[39*256 + int((index_7 >> 0).cast[DType.uint8]())] ^
                table[40*256 + int((index_6 >> 24).cast[DType.uint8]())] ^
                table[41*256 + int((index_6 >> 16).cast[DType.uint8]())] ^
                table[42*256 + int((index_6 >> 8).cast[DType.uint8]())] ^
                table[43*256 + int((index_6 >> 0).cast[DType.uint8]())] ^
                table[44*256 + int((index_5 >> 24).cast[DType.uint8]())] ^
                table[45*256 + int((index_5 >> 16).cast[DType.uint8]())] ^
                table[46*256 + int((index_5 >> 8).cast[DType.uint8]())] ^
                table[47*256 + int((index_5 >> 0).cast[DType.uint8]())] ^
                table[48*256 + int((index_4 >> 24).cast[DType.uint8]())] ^
                table[49*256 + int((index_4 >> 16).cast[DType.uint8]())] ^
                table[50*256 + int((index_4 >> 8).cast[DType.uint8]())] ^
                table[51*256 + int((index_4 >> 0).cast[DType.uint8]())] ^
                table[52*256 + int((index_3 >> 24).cast[DType.uint8]())] ^
                table[53*256 + int((index_3 >> 16).cast[DType.uint8]())] ^
                table[54*256 + int((index_3 >> 8).cast[DType.uint8]())] ^
                table[55*256 + int((index_3 >> 0).cast[DType.uint8]())] ^
                table[56*256 + int((index_2 >> 24).cast[DType.uint8]())] ^
                table[57*256 + int((index_2 >> 16).cast[DType.uint8]())] ^
                table[58*256 + int((index_2 >> 8).cast[DType.uint8]())] ^
                table[59*256 + int((index_2 >> 0).cast[DType.uint8]())] ^
                table[60*256 + int((index_1 >> 24).cast[DType.uint8]())] ^
                table[61*256 + int((index_1 >> 16).cast[DType.uint8]())] ^
                table[62*256 + int((index_1 >> 8).cast[DType.uint8]())] ^
                table[63*256 + int((index_1 >> 0).cast[DType.uint8]())] 
    
    for i in range(size*length, size*length + extra ):
        var index = (crc32 ^ data[i].cast[DType.uint32]()) & 0xff
        crc32 = table[int(index)] ^ (crc32 >> 8)


    return crc32^0xffffffff

In [ ]:
var little_endian_table_64_byte  = fill_table_n_byte[64]()
print(hex(CRC32(test_list)))
print(hex(CRC32_inv(test_list)))
print(hex(CRC32_table_64_byte(test_list, little_endian_table_64_byte)))

0x382aa34e
0x382aa34e
0x382aa34e


In [ ]:
fn run_32[data: List[SIMD[DType.uint8, 1]] ]():
    var a =  CRC32(data)
    benchmark.keep(a)


fn run_32_inv[data: List[SIMD[DType.uint8, 1]] ]():
    var a = CRC32_inv(data)
    benchmark.keep(a)


fn run_32_table[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table(data, table)
    benchmark.keep(a)


fn run_32_table_2_byte_2[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_2_byte_2(data, table)
    benchmark.keep(a)

fn run_32_table_4_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_4_byte(data, table)
    benchmark.keep(a)

fn run_32_table_8_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_8_byte(data, table)
    benchmark.keep(a)


fn run_32_table_16_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_16_byte(data, table)
    benchmark.keep(a)

fn run_32_table_32_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_32_byte(data, table)
    benchmark.keep(a)

fn run_32_table_64_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_64_byte(data, table)
    benchmark.keep(a)

fn bench():

    
    alias fill_size = 2**20
    alias g = UnsafePointer[SIMD[DType.uint8, 1]].alloc(fill_size)
    rand[DType.uint8](ptr =  g, size = fill_size)


    alias rand_list = List[SIMD[DType.uint8,1]](data = g, size = fill_size, capacity = fill_size)


    print(len(rand_list))

    var report = benchmark.run[run_32[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report)

    var report_2 = benchmark.run[run_32_inv[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_2)

 
    
    alias little_endian_table = fill_table_n_byte[1]()

    var report_3 = benchmark.run[run_32_table[rand_list, little_endian_table]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_3)

    alias little_endian_table_2_byte = fill_table_n_byte[2]()

    var report_4 = benchmark.run[run_32_table_2_byte_2[rand_list, little_endian_table_2_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_4)

    alias little_endian_table_4_byte = fill_table_n_byte[4]()

    var report_5 = benchmark.run[run_32_table_4_byte[rand_list, little_endian_table_4_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_5)

    alias little_endian_table_8_byte = fill_table_n_byte[8]()

    var report_6 = benchmark.run[run_32_table_8_byte[rand_list, little_endian_table_8_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_6)

    alias little_endian_table_16_byte = fill_table_n_byte[16]()

    var report_7 = benchmark.run[run_32_table_16_byte[rand_list, little_endian_table_16_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_7)

    alias little_endian_table_32_byte = fill_table_n_byte[32]()

    var report_8 = benchmark.run[run_32_table_32_byte[rand_list, little_endian_table_32_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_8)

    alias little_endian_table_64_byte = fill_table_n_byte[64]()

    var report_9 = benchmark.run[run_32_table_64_byte[rand_list, little_endian_table_64_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_9)

    print(100 * (report/report_2 -1))
    print(100 * (report_2/report_3 -1))
    print(100 * (report/report_3 -1))
    print(100 * (report/report_4 -1))
    print(100 * (report/report_5 -1))
    print(100 * (report/report_6 -1))
    print(100 * (report/report_7 -1))
    print(100 * (report/report_8 -1))
    print(100 * (report/report_9 -1))



    #report.print_full()
    #report_2.print_full()


bench()

1048576
8.0958940049999999
7.3598195150000008
2.1762406051159076
1.5418122323619632
0.73977778249999993
0.50607860449999997
0.2477206075
0.23779043850000001
0.32381718649999996
10.001257347409265
238.18960540018108
272.01281815844112
425.0894911241935
994.36836256974243
1499.730542451731
3168.1552361363392
3304.6339525127705
2400.1433964963439


In [ ]:
fn CRC32_table_48_byte(owned data: List[SIMD[DType.uint8, 1]], table: List[UInt32]) -> SIMD[DType.uint32, 1]:
    var crc32: UInt32 = 0xffffffff

    var size = 48

    var length = len(data)//size
    var extra = len(data) % size



    for i in range(start = 0, end = length*size, step = size):
        
        var val_1: UInt32 = (data[i + 3].cast[DType.uint32]() << 24) | 
                            (data[i + 2].cast[DType.uint32]() << 16) | 
                            (data[i + 1].cast[DType.uint32]() << 8) | 
                             data[i + 0].cast[DType.uint32]()

        var val_2: UInt32 = (data[i + 7].cast[DType.uint32]() << 24) | 
                            (data[i + 6].cast[DType.uint32]() << 16) | 
                            (data[i + 5].cast[DType.uint32]() << 8) | 
                             data[i + 4].cast[DType.uint32]()

        var val_3: UInt32 = (data[i + 11].cast[DType.uint32]() << 24) | 
                            (data[i + 10].cast[DType.uint32]() << 16) | 
                            (data[i + 9].cast[DType.uint32]() << 8) | 
                             data[i + 8].cast[DType.uint32]()

        var val_4: UInt32 = (data[i + 15].cast[DType.uint32]() << 24) | 
                            (data[i + 14].cast[DType.uint32]() << 16) | 
                            (data[i + 13].cast[DType.uint32]() << 8) | 
                             data[i + 12].cast[DType.uint32]()

        var val_5: UInt32 = (data[i + 19].cast[DType.uint32]() << 24) | 
                            (data[i + 18].cast[DType.uint32]() << 16) | 
                            (data[i + 17].cast[DType.uint32]() << 8) | 
                             data[i + 16].cast[DType.uint32]()

        var val_6: UInt32 = (data[i + 23].cast[DType.uint32]() << 24) | 
                            (data[i + 22].cast[DType.uint32]() << 16) | 
                            (data[i + 21].cast[DType.uint32]() << 8) | 
                             data[i + 20].cast[DType.uint32]()

        var val_7: UInt32 = (data[i + 27].cast[DType.uint32]() << 24) | 
                            (data[i + 26].cast[DType.uint32]() << 16) | 
                            (data[i + 25].cast[DType.uint32]() << 8) | 
                             data[i + 24].cast[DType.uint32]()

        var val_8: UInt32 = (data[i + 31].cast[DType.uint32]() << 24) | 
                            (data[i + 30].cast[DType.uint32]() << 16) | 
                            (data[i + 29].cast[DType.uint32]() << 8) | 
                             data[i + 28].cast[DType.uint32]()


        var val_9: UInt32 = (data[i + 35].cast[DType.uint32]() << 24) | 
                            (data[i + 34].cast[DType.uint32]() << 16) | 
                            (data[i + 33].cast[DType.uint32]() << 8) | 
                             data[i + 32].cast[DType.uint32]()

        var val_10: UInt32 =(data[i + 39].cast[DType.uint32]() << 24) | 
                            (data[i + 38].cast[DType.uint32]() << 16) | 
                            (data[i + 37].cast[DType.uint32]() << 8) | 
                             data[i + 36].cast[DType.uint32]()

        var val_11: UInt32 =(data[i + 43].cast[DType.uint32]() << 24) | 
                            (data[i + 42].cast[DType.uint32]() << 16) | 
                            (data[i + 41].cast[DType.uint32]() << 8) | 
                             data[i + 40].cast[DType.uint32]()

        var val_12: UInt32 =(data[i + 47].cast[DType.uint32]() << 24) | 
                            (data[i + 46].cast[DType.uint32]() << 16) | 
                            (data[i + 45].cast[DType.uint32]() << 8) | 
                             data[i + 44].cast[DType.uint32]()



        var index_1 = crc32 ^ val_1.cast[DType.uint32]()
        var index_2 = val_2.cast[DType.uint32]()
        var index_3 = val_3.cast[DType.uint32]()
        var index_4 = val_4.cast[DType.uint32]()
        var index_5 = val_5.cast[DType.uint32]()
        var index_6 = val_6.cast[DType.uint32]()
        var index_7 = val_7.cast[DType.uint32]()
        var index_8 = val_8.cast[DType.uint32]()
        var index_9 = val_9.cast[DType.uint32]()
        var index_10 = val_10.cast[DType.uint32]()
        var index_11 = val_11.cast[DType.uint32]()
        var index_12 = val_12.cast[DType.uint32]()

        crc32 = table[0*256 + int((index_12 >> 24).cast[DType.uint8]())] ^
                table[1*256 + int((index_12 >> 16).cast[DType.uint8]())] ^
                table[2*256 + int((index_12 >> 8).cast[DType.uint8]())] ^
                table[3*256 + int((index_12 >> 0).cast[DType.uint8]())] ^
                table[4*256 + int((index_11 >> 24).cast[DType.uint8]())] ^
                table[5*256 + int((index_11 >> 16).cast[DType.uint8]())] ^
                table[6*256 + int((index_11 >> 8).cast[DType.uint8]())] ^
                table[7*256 + int((index_11 >> 0).cast[DType.uint8]())] ^
                table[8*256 + int((index_10 >> 24).cast[DType.uint8]())] ^
                table[9*256 + int((index_10 >> 16).cast[DType.uint8]())] ^
                table[10*256 + int((index_10 >> 8).cast[DType.uint8]())] ^
                table[11*256 + int((index_10 >> 0).cast[DType.uint8]())] ^
                table[12*256 + int((index_9 >> 24).cast[DType.uint8]())] ^
                table[13*256 + int((index_9 >> 16).cast[DType.uint8]())] ^
                table[14*256 + int((index_9 >> 8).cast[DType.uint8]())] ^
                table[15*256 + int((index_9 >> 0).cast[DType.uint8]())] ^
                table[16*256 + int((index_8 >> 24).cast[DType.uint8]())] ^
                table[17*256 + int((index_8 >> 16).cast[DType.uint8]())] ^
                table[18*256 + int((index_8 >> 8).cast[DType.uint8]())] ^
                table[19*256 + int((index_8 >> 0).cast[DType.uint8]())] ^
                table[20*256 + int((index_7 >> 24).cast[DType.uint8]())] ^
                table[21*256 + int((index_7 >> 16).cast[DType.uint8]())] ^
                table[22*256 + int((index_7 >> 8).cast[DType.uint8]())] ^
                table[23*256 + int((index_7 >> 0).cast[DType.uint8]())] ^
                table[24*256 + int((index_6 >> 24).cast[DType.uint8]())] ^
                table[25*256 + int((index_6 >> 16).cast[DType.uint8]())] ^
                table[26*256 + int((index_6 >> 8).cast[DType.uint8]())] ^
                table[27*256 + int((index_6 >> 0).cast[DType.uint8]())] ^
                table[28*256 + int((index_5 >> 24).cast[DType.uint8]())] ^
                table[29*256 + int((index_5 >> 16).cast[DType.uint8]())] ^
                table[30*256 + int((index_5 >> 8).cast[DType.uint8]())] ^
                table[31*256 + int((index_5 >> 0).cast[DType.uint8]())] ^
                table[32*256 + int((index_4 >> 24).cast[DType.uint8]())] ^
                table[33*256 + int((index_4 >> 16).cast[DType.uint8]())] ^
                table[34*256 + int((index_4 >> 8).cast[DType.uint8]())] ^
                table[35*256 + int((index_4 >> 0).cast[DType.uint8]())] ^
                table[36*256 + int((index_3 >> 24).cast[DType.uint8]())] ^
                table[37*256 + int((index_3 >> 16).cast[DType.uint8]())] ^
                table[38*256 + int((index_3 >> 8).cast[DType.uint8]())] ^
                table[39*256 + int((index_3 >> 0).cast[DType.uint8]())] ^
                table[40*256 + int((index_2 >> 24).cast[DType.uint8]())] ^
                table[41*256 + int((index_2 >> 16).cast[DType.uint8]())] ^
                table[42*256 + int((index_2 >> 8).cast[DType.uint8]())] ^
                table[43*256 + int((index_2 >> 0).cast[DType.uint8]())] ^
                table[44*256 + int((index_1 >> 24).cast[DType.uint8]())] ^
                table[45*256 + int((index_1 >> 16).cast[DType.uint8]())] ^
                table[46*256 + int((index_1 >> 8).cast[DType.uint8]())] ^
                table[47*256 + int((index_1 >> 0).cast[DType.uint8]())] 
                
    for i in range(size*length, size*length + extra ):
        var index = (crc32 ^ data[i].cast[DType.uint32]()) & 0xff
        crc32 = table[int(index)] ^ (crc32 >> 8)


    return crc32^0xffffffff

In [ ]:
var little_endian_table_48_byte  = fill_table_n_byte[48]()
print(hex(CRC32(test_list)))
print(hex(CRC32_inv(test_list)))
print(hex(CRC32_table_48_byte(test_list, little_endian_table_48_byte)))



error: Expression [23]:1:36: use of unknown declaration 'fill_table_n_byte'
var little_endian_table_48_byte  = fill_table_n_byte[48]()
                                   ^~~~~~~~~~~~~~~~~
error: Expression [23]:2:11: use of unknown declaration 'CRC32'
print(hex(CRC32(test_list)))
          ^~~~~
error: Expression [23]:3:11: use of unknown declaration 'CRC32_inv'
print(hex(CRC32_inv(test_list)))
          ^~~~~~~~~
error: Expression [23]:4:11: use of unknown declaration 'CRC32_table_48_byte'
print(hex(CRC32_table_48_byte(test_list, little_endian_table_48_byte)))
          ^~~~~~~~~~~~~~~~~~~


expression failed to parse (no further compiler diagnostics)

In [ ]:
fn run_32[data: List[SIMD[DType.uint8, 1]] ]():
    var a =  CRC32(data)
    benchmark.keep(a)


fn run_32_inv[data: List[SIMD[DType.uint8, 1]] ]():
    var a = CRC32_inv(data)
    benchmark.keep(a)


fn run_32_table[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table(data, table)
    benchmark.keep(a)


fn run_32_table_2_byte_2[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_2_byte_2(data, table)
    benchmark.keep(a)

fn run_32_table_4_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_4_byte(data, table)
    benchmark.keep(a)

fn run_32_table_8_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_8_byte(data, table)
    benchmark.keep(a)


fn run_32_table_16_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_16_byte(data, table)
    benchmark.keep(a)

fn run_32_table_32_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_32_byte(data, table)
    benchmark.keep(a)

fn run_32_table_48_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_48_byte(data, table)
    benchmark.keep(a)

fn run_32_table_64_byte[data: List[SIMD[DType.uint8, 1]], table: List[UInt32]]():
    var a = CRC32_table_64_byte(data, table)
    benchmark.keep(a)

fn bench():

    
    alias fill_size = 2**20
    alias g = UnsafePointer[SIMD[DType.uint8, 1]].alloc(fill_size)
    rand[DType.uint8](ptr =  g, size = fill_size)


    alias rand_list = List[SIMD[DType.uint8,1]](data = g, size = fill_size, capacity = fill_size)


    print(len(rand_list))

    var report = benchmark.run[run_32[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report)

    var report_2 = benchmark.run[run_32_inv[rand_list]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_2)

 
    
    alias little_endian_table = fill_table_n_byte[1]()

    var report_3 = benchmark.run[run_32_table[rand_list, little_endian_table]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_3)

    alias little_endian_table_2_byte = fill_table_n_byte[2]()

    var report_4 = benchmark.run[run_32_table_2_byte_2[rand_list, little_endian_table_2_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_4)

    alias little_endian_table_4_byte = fill_table_n_byte[4]()

    var report_5 = benchmark.run[run_32_table_4_byte[rand_list, little_endian_table_4_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_5)

    alias little_endian_table_8_byte = fill_table_n_byte[8]()

    var report_6 = benchmark.run[run_32_table_8_byte[rand_list, little_endian_table_8_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_6)

    alias little_endian_table_16_byte = fill_table_n_byte[16]()

    var report_7 = benchmark.run[run_32_table_16_byte[rand_list, little_endian_table_16_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_7)

    alias little_endian_table_32_byte = fill_table_n_byte[32]()

    var report_8 = benchmark.run[run_32_table_32_byte[rand_list, little_endian_table_32_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_8)

    alias little_endian_table_48_byte = fill_table_n_byte[48]()

    var report_9 = benchmark.run[run_32_table_48_byte[rand_list, little_endian_table_48_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_9)

    alias little_endian_table_64_byte = fill_table_n_byte[64]()

    var report_10 = benchmark.run[run_32_table_64_byte[rand_list, little_endian_table_64_byte]](max_runtime_secs=0.5
    ).mean(benchmark.Unit.ms)
    print(report_10)

    print(100 * (report/report_2 -1))
    print(100 * (report_2/report_3 -1))
    print(100 * (report/report_3 -1))
    print(100 * (report/report_4 -1))
    print(100 * (report/report_5 -1))
    print(100 * (report/report_6 -1))
    print(100 * (report/report_7 -1))
    print(100 * (report/report_8 -1))
    print(100 * (report/report_9 -1))
    print(100 * (report/report_10 -1))




    #report.print_full()
    #report_2.print_full()


bench()

1048576
8.15321797
7.0219428749999997
1.9393683111291633
1.3442710239154616
0.70748446850000002
0.48288434749999998
0.234955997
0.18959147131244575
0.40115316100000004
0.25623588050000001
16.110571036224798
262.07371414208558
320.40585706244372
506.51593502715701
1052.4235983987542
1588.4411375541636
3370.104221259779
4200.4138918061035
1932.4451512922267
3081.9189233336115


In [ ]:
%%python
from timeit import timeit
import numpy as np
import zlib


py_rand_list = np.random.randint(0, 255, size=2**18, dtype=np.uint8)
print(hex(zlib.crc32(py_rand_list)))

secs = timeit(lambda: zlib.crc32(py_rand_list), number=100)/100

print(secs*100)

0x5ada5791
0.021449088002555072


In [ ]:
print(secs*100)

0.021655022050254047


In [ ]:
fn fill_table_n_byte_simd[n: Int]() -> DTypePointer[DType.uint32]:

    #alias size = 256*n
    #var table = SIMD[DType.uint32, size](0)
    var table = DTypePointer[DType.uint32]()
    table = table.alloc(256*n, alignment=64)

    for i in range(256*n):

        if i < 256: 
            var key = UInt8(i)
            var crc32 = key.cast[DType.uint32]()
            for i in range(8):
                if crc32 & 1 != 0:
                    crc32 = (crc32 >> 1) ^ 0xedb88320
                else:
                    crc32 = crc32 >> 1

            table[i] = crc32
        else:
            var crc32 = table[i-256]
            var index = int(crc32.cast[DType.uint8]())
            table[i] = (crc32 >> 8) ^ table[index]
            
    return table


In [ ]:
alias let_simd = fill_table_n_byte_simd[32]()

In [ ]:
print(let_simd.is_aligned[128]())

True


In [ ]:
def main():
    var a: UInt32 = 129

    print(int(a.cast[DType.uint8]()))

main()

-127


In [ ]:
var a: UInt32 = 129

print(int(a.cast[DType.uint8]()))

129


In [ ]:
def main():
  var n = UInt8(128)
  print(n)       # prints 128
  print(int(n))  # prints 128

main()

128
128
